Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

In [10]:
import gensim
import json
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [11]:
with open('habr_texts.txt', encoding='utf-8') as file:
    source = file.read().splitlines()

Решил оставить только слова написанные кирилицей, поскольку в статьях много кода, который никак не отделён от основного текста. Попытки оставить только "полезные" слова вроде названий языков программирования, фреймворков и прочего непропорционально усложняют решение, при этом, по ощущения, полезнее убрать код.

*P.S. Забыл, что в регулярных выражениях нужно отдельно задавать букву ё, что повляило на все дальнейшие темы. Не стал уже всё заново переделывать, но урок для себя извлёк.*

In [12]:
stop_words = set(stopwords.words('russian')) | {'gt',}

def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'{[^}]+}', ' ', text)
    text = re.sub(r'\&[a-z]+\;', ' ', text)
    text = text.replace('\t', ' ')
    return text

def tokenize(text):
    return re.findall(r'[А-Яа-я]+', text)

# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stop_words]
        norm_texts.append(norm_words)
        
    return norm_texts


2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

In [13]:
texts = opt_normalize([tokenize(clean_text(text)) for text in source])

In [14]:
ph = gensim.models.Phrases(texts, scoring='npmi', min_count=8, threshold=0.2)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

In [15]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [16]:
dictionary.filter_extremes(no_above=0.1, no_below=15)
dictionary.compactify()

In [27]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

4) постройте несколько LDA моделей (переберите количество тем, можете поменять alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

In [18]:
lda_1 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=2, alpha='asymmetric')
lda_2 = gensim.models.LdaModel(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='auto')
lda_3 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=2, num_topics=15, passes=4)

In [19]:
lda_1.print_topics(10)

[(0,
  '0.002*"бот" + 0.002*"домен" + 0.002*"плагин" + 0.002*"звук" + 0.002*"ключ" + 0.002*"игрок" + 0.001*"телефон" + 0.001*"папка" + 0.001*"аппарат" + 0.001*"драйвер"'),
 (1,
  '0.005*"камера" + 0.002*"смартфон" + 0.002*"датчик" + 0.002*"робот" + 0.002*"заказчик" + 0.002*"массив" + 0.001*"транзакция" + 0.001*"шаблон" + 0.001*"контейнер" + 0.001*"автомобиль"'),
 (2,
  '0.004*"сигнал" + 0.003*"процессор" + 0.002*"ядро" + 0.002*"диск" + 0.001*"энергия" + 0.001*"робот" + 0.001*"стандарт" + 0.001*"частота" + 0.001*"транзистор" + 0.001*"звук"'),
 (3,
  '0.002*"ядро" + 0.002*"трафик" + 0.002*"игрок" + 0.001*"хост" + 0.001*"перевод" + 0.001*"блокчейн" + 0.001*"ключ" + 0.001*"участник" + 0.001*"массив" + 0.001*"пароль"'),
 (4,
  '0.002*"атака" + 0.002*"услуга" + 0.002*"участник" + 0.002*"доклад" + 0.002*"матрица" + 0.002*"уязвимость" + 0.002*"трафик" + 0.002*"принтер" + 0.002*"вектор" + 0.002*"браузер"'),
 (5,
  '0.003*"браузер" + 0.002*"сертификат" + 0.002*"ключ" + 0.002*"токен" + 0.001*"раз

In [20]:
lda_2.print_topics(5)

[(0,
  '0.007*"игрок" + 0.003*"яркость" + 0.003*"удача" + 0.003*"клетка" + 0.003*"стикер" + 0.002*"цвет" + 0.002*"стандарт" + 0.002*"закон" + 0.002*"цифровой" + 0.002*"двигатель"'),
 (1,
  '0.005*"вакансия" + 0.005*"температура" + 0.004*"прибор" + 0.003*"головка" + 0.003*"датчик" + 0.003*"производство" + 0.003*"камера" + 0.003*"печать" + 0.003*"сигнал" + 0.003*"аккумулятор"'),
 (2,
  '0.004*"массив" + 0.003*"браузер" + 0.003*"атрибут" + 0.003*"цикл" + 0.002*"шаблон" + 0.002*"контейнер" + 0.002*"контекст" + 0.002*"переменный" + 0.002*"ключ" + 0.002*"узел"'),
 (3,
  '0.006*"игрок" + 0.005*"боль" + 0.005*"студент" + 0.004*"мозг" + 0.003*"курс" + 0.003*"программирование" + 0.003*"аудитория" + 0.003*"играть" + 0.003*"обучение" + 0.003*"браузер"'),
 (4,
  '0.008*"услуга" + 0.005*"заказчик" + 0.003*"продажа" + 0.003*"реклама" + 0.003*"трафик" + 0.002*"книга" + 0.002*"инвестиция" + 0.002*"российский" + 0.002*"покупка" + 0.002*"партнёр"')]

In [21]:
lda_3.print_topics(15)

[(0,
  '0.007*"диск" + 0.005*"вакансия" + 0.003*"генератор" + 0.003*"массив" + 0.003*"транзакция" + 0.003*"спутник" + 0.002*"выражение" + 0.002*"переменный" + 0.002*"полоса" + 0.002*"микросхема"'),
 (1,
  '0.003*"пациент" + 0.003*"оператор" + 0.002*"врач" + 0.002*"болезнь" + 0.002*"мозг" + 0.002*"глаз" + 0.002*"продажа" + 0.002*"китай" + 0.002*"стандарт" + 0.002*"заболевание"'),
 (2,
  '0.004*"доклад" + 0.003*"книга" + 0.003*"конференция" + 0.003*"вселенная" + 0.002*"курс" + 0.002*"человек_который" + 0.002*"энергия" + 0.002*"галактика" + 0.002*"теория" + 0.002*"участник"'),
 (3,
  '0.004*"контейнер" + 0.004*"ядро" + 0.003*"переменный" + 0.003*"кластер" + 0.003*"массив" + 0.003*"драйвер" + 0.003*"узел" + 0.002*"блокчейн" + 0.002*"индекс" + 0.002*"ключ"'),
 (4,
  '0.005*"земля" + 0.004*"аппарат" + 0.004*"атака" + 0.004*"дата_центр" + 0.004*"услуга" + 0.004*"марс" + 0.004*"спутник" + 0.004*"орбита" + 0.003*"астероид" + 0.003*"провайдер"'),
 (5,
  '0.003*"уведомление" + 0.003*"трафик" + 0.

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

В целом, у всех моделей темы, конечно, не идеальные, но кажется, что модель с большим количество лучше их поделила:

Машинное обучение
```
(9,
  '0.003*"вектор" + 0.003*"обучение" + 0.003*"печать" + 0.002*"матрица" + 0.002*"нейронный_сеть" + 0.002*"признак" + 0.002*"слой" + 0.002*"набор_дать" + 0.002*"нейросеть" + 0.002*"нейрон"'),
```
Процесс запуска ОС
```
 (12,
  '0.005*"номер" + 0.004*"ядро" + 0.003*"регистр" + 0.003*"битый" + 0.003*"инструкция" + 0.003*"сигнал" + 0.003*"процессор" + 0.002*"датчик" + 0.002*"частота" + 0.002*"ветка"'),
```
Что-то про безопасность в сети
```
(14,
  '0.006*"браузер" + 0.005*"сигнал" + 0.003*"плагин" + 0.003*"вкладка" + 0.003*"транзакция" + 0.002*"глаз" + 0.002*"домен" + 0.002*"окно" + 0.002*"меню" + 0.002*"репозиторий"')
```

Бонус
```
 (3,
  '0.006*"игрок" + 0.005*"боль" + 0.005*"студент" + 0.004*"мозг" + 0.003*"курс" + 0.003*"программирование" + 0.003*"аудитория" + 0.003*"играть" + 0.003*"обучение" + 0.003*"браузер"'),
```

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

In [29]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]

7) повторите пункт 4 на преобразованном корпусе;

In [23]:
lda_1_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=10, passes=2, alpha='asymmetric')
lda_2_tfidf = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='auto')
lda_3_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=2, num_topics=15, passes=4)

In [24]:
lda_1_tfidf.print_topics(10)

[(0,
  '0.001*"игрок" + 0.001*"браузер" + 0.001*"бот" + 0.001*"книга" + 0.001*"доклад" + 0.001*"ядро" + 0.001*"камера" + 0.001*"ключ" + 0.001*"телефон" + 0.001*"смартфон"'),
 (1,
  '0.001*"доклад" + 0.001*"игрок" + 0.001*"конференция" + 0.001*"станция" + 0.001*"диск" + 0.001*"камера" + 0.001*"смартфон" + 0.001*"плагин" + 0.001*"массив" + 0.001*"ия"'),
 (2,
  '0.001*"бот" + 0.001*"браузер" + 0.001*"ключ" + 0.000*"указатель" + 0.000*"телефон" + 0.000*"виртуальный_машина" + 0.000*"переменный" + 0.000*"ядро" + 0.000*"ос" + 0.000*"оператор"'),
 (3,
  '0.001*"контейнер" + 0.001*"массив" + 0.001*"схд" + 0.000*"домен" + 0.000*"блокчейн" + 0.000*"заказчик" + 0.000*"шлюз" + 0.000*"протокол" + 0.000*"компилятор" + 0.000*"хост"'),
 (4,
  '0.001*"узел" + 0.000*"адаптер" + 0.000*"книга" + 0.000*"процессор" + 0.000*"регистратор" + 0.000*"письмо" + 0.000*"игрок" + 0.000*"социальный_сеть" + 0.000*"проектор" + 0.000*"яркость"'),
 (5,
  '0.000*"символ" + 0.000*"автомобиль" + 0.000*"магазин" + 0.000*"вяче

In [25]:
lda_2_tfidf.print_topics(5)

[(0,
  '0.000*"ротация" + 0.000*"смартфон" + 0.000*"фонарь" + 0.000*"блок_блок" + 0.000*"датчик" + 0.000*"шлюз" + 0.000*"принтер" + 0.000*"сбрасывать" + 0.000*"счётчик" + 0.000*"мм"'),
 (1,
  '0.004*"контейнер" + 0.003*"массив" + 0.003*"конвертер" + 0.003*"цикл" + 0.002*"репликация" + 0.002*"токен" + 0.002*"кластер" + 0.002*"атрибут" + 0.002*"нода" + 0.002*"логин"'),
 (2,
  '0.002*"услуга" + 0.001*"браузер" + 0.001*"бот" + 0.001*"контекст" + 0.001*"заказчик" + 0.001*"трафик" + 0.001*"контент" + 0.001*"студент" + 0.001*"инцидент" + 0.001*"веб"'),
 (3,
  '0.001*"прибор" + 0.001*"игрок" + 0.001*"вакансия" + 0.001*"аккумулятор" + 0.001*"мозг" + 0.001*"стикер" + 0.001*"боль" + 0.001*"температура" + 0.001*"дефект" + 0.001*"дом"'),
 (4,
  '0.007*"доклад" + 0.003*"протон" + 0.003*"мероприятие" + 0.002*"февраль" + 0.002*"неформальный" + 0.002*"роскосмос" + 0.002*"пообщаться" + 0.002*"двигатель" + 0.002*"докладчик" + 0.001*"сколько_часы"')]

In [26]:
lda_3_tfidf.print_topics(15)

[(0,
  '0.002*"доклад" + 0.002*"пациент" + 0.001*"книга" + 0.001*"клетка" + 0.001*"анимация" + 0.001*"глаз" + 0.001*"конференция" + 0.001*"робот" + 0.001*"ген" + 0.001*"игрок"'),
 (1,
  '0.001*"диск" + 0.001*"контроллер" + 0.001*"сигнал" + 0.001*"ключ" + 0.001*"протокол" + 0.001*"массив" + 0.001*"шаблон" + 0.001*"камера" + 0.001*"стандарт" + 0.001*"байт"'),
 (2,
  '0.001*"указатель" + 0.001*"ключ" + 0.001*"принтер" + 0.001*"дата_центр" + 0.001*"репозиторий" + 0.001*"переменный" + 0.001*"доклад" + 0.001*"уведомление" + 0.001*"массив" + 0.001*"лекция"'),
 (3,
  '0.001*"телефон" + 0.001*"игрок" + 0.001*"смартфон" + 0.001*"камера" + 0.001*"услуга" + 0.001*"контент" + 0.001*"бот" + 0.001*"лекция" + 0.001*"аудитория" + 0.001*"уязвимость"'),
 (4,
  '0.002*"ия" + 0.001*"вирус" + 0.001*"плагин" + 0.001*"станок" + 0.001*"битрикс" + 0.001*"книга" + 0.001*"атака" + 0.001*"ключ" + 0.001*"удалённый" + 0.001*"выкуп"'),
 (5,
  '0.004*"галактика" + 0.002*"вселенная" + 0.001*"материя" + 0.001*"зв_зд" + 

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

Любопытно, что тем про машинное обучение вообще пропала. *(рассматриваю модель 3 как лучшую)*

Более явно выделилась тема про полеты в космос
```
 (11,
  '0.003*"спутник" + 0.003*"марс" + 0.002*"наса" + 0.002*"орбита" + 0.002*"ракета" + 0.002*"луна" + 0.002*"земля" + 0.002*"космос" + 0.001*"пуск" + 0.001*"астронавт"'),
```

В тема про блочейн появилась криптовалюта, но при этом также появился беспилотный автомобиль. Видимо смешались 2 темы.
```
 (13,
  '0.002*"блокчейн" + 0.001*"плагин" + 0.001*"камера" + 0.001*"шифрование" + 0.001*"пароль" + 0.001*"криптовалюта" + 0.001*"фич" + 0.001*"книга" + 0.001*"смартфон" + 0.001*"беспилотный_автомобиль"'),
```
Появилась еще одна более общая тема про космос, в которую влезли процессор и диск :(
```
 (5,
  '0.004*"галактика" + 0.002*"вселенная" + 0.001*"материя" + 0.001*"зв_зд" + 0.001*"солнце" + 0.001*"зв_зды" + 0.001*"процессор" + 0.001*"миллиард_год" + 0.001*"звезда" + 0.001*"диск"'),
```

Можно резюмировать, что ни один за наборов параметров не дал заметно хороших результатов. Использование tfidf действительно изменило темы, но я не могу сказать, что они стали объективно лучше.

Использование tfidf даже увеличило перплексию

In [31]:
print(lda_3.log_perplexity(corpus[:10000]))
print(lda_3_tfidf.log_perplexity(corpus_tfidf[:10000]))

-8.952846011807756
-12.603523671209263


9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

In [33]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [47]:
str_texts = [' '.join(text) for text in texts]

In [55]:
vectorizer = TfidfVectorizer(max_features=800, min_df=12, max_df=0.4, ngram_range=(1,2))
X = vectorizer.fit_transform(str_texts)

In [56]:
model = NMF(n_components=30)

In [57]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [58]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [59]:
feat_names = vectorizer.get_feature_names()

In [60]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 вс  книга  ещ  метр
1 файл  папка  скрипт  строка
2 игра  игрок  игровой  играть
3 команда  продукт  разработчик  разработка
4 память  процессор  диск  ядро
5 приложение  мобильный  разработка  платформа
6 рынок  бизнес  сервис  технология
7 сайт  реклама  страница  контент
8 учёный  мозг  исследование  научный
9 уязвимость  атака  безопасность  пароль
10 браузер  веб  версия  страница
11 рубль  товар  цена  скидка
12 доклад  конференция  разработчик  видео
13 код  функция  ошибка  переменный
14 сигнал  частота  диапазон  канал
15 тест  тестирование  тестовый  библиотека
16 бот  сообщение  канал  сервис
17 объект  класс  метод  тип
18 сеть  интернет  трафик  связь
19 язык  программирование  программист  программа
20 дата центр  центр  дата  оборудование
21 звук  материал  частота  читать
22 камера  видео  изображение  смартфон
23 устройство  смартфон  телефон  мобильный
24 виртуальный  машина  автомобиль  реальность
25 сервер  запрос  база  сервис
26 робот  датчик  ребёнок  обучение


In [62]:
vectorizer = TfidfVectorizer(max_features=600, min_df=6, max_df=0.2, ngram_range=(1,3))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=40)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 бизнес  сотрудник  ит  заказчик
1 игра  игрок  игровой  играть
2 строка  переменный  массив  вызов
3 энергия  свет  температура  теория
4 диск  хранение  хранилище  кластер
5 камера  видео  смартфон  кадр
6 ключ  сертификат  пароль  открытый
7 доклад  конференция  участник  видео
8 класс  свойство  вызов  логика
9 тест  тестирование  тестовый  баг
10 запись  таблица  чтение  база дать
11 браузер  страница  веб  реклама
12 уязвимость  атака  безопасность  защита
13 дата центр  центр  дата  оборудование
14 бот  канал  аудитория  текст
15 книга  часы  читать  профессиональный
16 память  процессор  ядро  гб
17 запрос  таблица  база дать  выполнение
18 виртуальный  машина  автомобиль  реальность
19 печать  мм  производство  температура
20 адрес  домен  письмо  пароль
21 модуль  контроллер  папка  подключение
22 робот  ребёнок  датчик  движение
23 космический  аппарат  земля  станция
24 изображение  алгоритм  обучение  слой
25 мобильный  телефон  смартфон  карта
26 экран  кнопка  текст  ди

In [67]:
vectorizer = CountVectorizer(max_features=70, min_df=15, max_df=0.4, ngram_range=(1,4))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=20)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 код  ошибка  программа  память
1 игра  разработчик  разработка  уровень
2 тип  класс  метод  интерфейс
3 устройство  управление  уровень  доступ
4 значение  день  команда  параметр
5 вс  ещ  говорить  знать
6 файл  добавить  параметр  создать
7 приложение  ваш  разработчик  версия
8 сайт  страница  сервис  продукт
9 сервер  клиент  сервис  ваш
10 сеть  интерфейс  технология  использоваться
11 функция  параметр  уровень  список
12 модель  метод  уровень  сервис
13 язык  программа  технология  разработка
14 команда  продукт  разработка  разработчик
15 объект  метод  класс  инструмент
16 элемент  страница  список  ошибка
17 запрос  база  сервис  ошибка
18 точка  доступ  вариант  место
19 метр  далее  память  управление


10) в отдельной ячейке напечатайте темы лучшей NMF модели, сравните их с теми, что получились в LDA.

*первая модель*

Снова тема про безопасность в сети (и не только)
```
9 уязвимость  атака  безопасность  пароль
```
Надо думать, мобильная разработка
```
5 приложение  мобильный  разработка  платформа
```
Опять же космос
```
28 космический  аппарат  земля  станция
```


Как бонус: 
```
24 виртуальный  машина  автомобиль  реальность
```
Опять откуда-то вылезает автомобиль. Может быть что-то связанное с машинами на автопилоте?

Можно резюмировать, что NMF выделяет более общие темы, чем LDA. Определенно сложнее понять о чем идет речь. При этом темы разбиты существенно более четко. Такого явного склеивания двух разных тем не наблюдается. Какой-то существенной разницы между CountVectorizer и TfidfVectorizer я не вижу – и там, и там вылезают стоп-слова, которые я забыл отсеять.